In [1]:
#!/usr/bin/env python3
"""
KAORU BRIDGE v9.0 - TRUE TIP INJECTION
Obtiene el tip REAL de la cadena (~875,000) y construye sobre él
"""

import hashlib
import struct
import socket
import time
import random
import select
from typing import List, Tuple, Dict, Optional
from dataclasses import dataclass
from collections import defaultdict
from datetime import datetime

@dataclass
class BlockHeader:
    version: int
    prev_hash: bytes
    merkle_root: bytes
    timestamp: int
    bits: int
    nonce: int

    def serialize(self) -> bytes:
        return (struct.pack('<I', self.version) + self.prev_hash +
                self.merkle_root + struct.pack('<I', self.timestamp) +
                struct.pack('<I', self.bits) + struct.pack('<I', self.nonce))

    def hash(self) -> bytes:
        return hashlib.sha256(hashlib.sha256(self.serialize()).digest()).digest()

    def hash_hex(self) -> str:
        return self.hash()[::-1].hex()

@dataclass
class Block:
    header: BlockHeader
    coinbase_script: bytes = b''

    def serialize(self) -> bytes:
        data = self.header.serialize()
        data += b'\x01'
        data += self._coinbase()
        return data

    def _coinbase(self) -> bytes:
        tx = struct.pack('<I', 1)
        tx += b'\x01' + bytes(32) + struct.pack('<I', 0xffffffff)
        script = self.coinbase_script or b'\x04KAORU'
        tx += bytes([len(script)]) + script + struct.pack('<I', 0xffffffff)
        tx += b'\x01' + struct.pack('<Q', 50 * 10**8)
        tx += b'\x19\x76\xa9\x14' + bytes(20) + b'\x88\xac'
        tx += struct.pack('<I', 0)
        return tx


class KaoruBridgeV9:
    """
    v9.0 - TRUE TIP INJECTION
    Sincroniza hasta el tip real y construye sobre él
    """

    MAINNET_MAGIC = bytes.fromhex('f9beb4d9')
    MSG_BLOCK = 2

    def __init__(self):
        self.magic = self.MAINNET_MAGIC
        self.our_blocks: List[Block] = []
        self.stats = defaultdict(int)
        self.all_headers: List[bytes] = []
        self.tip_header: Optional[bytes] = None
        self.tip_height: int = 0

    @staticmethod
    def double_sha256(data: bytes) -> bytes:
        return hashlib.sha256(hashlib.sha256(data).digest()).digest()

    def log(self, msg: str):
        t = datetime.now().strftime("%H:%M:%S.%f")[:-3]
        print(f"      [{t}] {msg}")

    def msg(self, cmd: str, payload: bytes) -> bytes:
        c = cmd.encode().ljust(12, b'\x00')
        checksum = self.double_sha256(payload)[:4]
        return self.magic + c + struct.pack('<I', len(payload)) + checksum + payload

    def version_payload(self, ip: str) -> bytes:
        p = struct.pack('<i', 70016)
        p += struct.pack('<Q', 1037)
        p += struct.pack('<q', int(time.time()))
        p += struct.pack('<Q', 1) + b'\x00'*10 + b'\xff\xff' + socket.inet_aton(ip) + struct.pack('>H', 8333)
        p += struct.pack('<Q', 1037) + b'\x00'*10 + b'\xff\xff' + b'\x00'*4 + struct.pack('>H', 8333)
        p += struct.pack('<Q', random.randint(0, 2**64-1))
        p += b'\x10/KaoruBridge:9.0/'
        p += struct.pack('<i', 900000)
        p += b'\x01'
        return p

    def getheaders_payload(self, block_locator: List[bytes]) -> bytes:
        p = struct.pack('<I', 70016)
        p += bytes([len(block_locator)])
        for h in block_locator:
            p += h
        p += bytes(32)  # stop hash
        return p

    def read_varint(self, data: bytes, offset: int) -> Tuple[int, int]:
        first = data[offset]
        if first < 0xfd:
            return first, offset + 1
        elif first == 0xfd:
            return struct.unpack('<H', data[offset+1:offset+3])[0], offset + 3
        elif first == 0xfe:
            return struct.unpack('<I', data[offset+1:offset+5])[0], offset + 5
        else:
            return struct.unpack('<Q', data[offset+1:offset+9])[0], offset + 9

    def parse_msgs(self, data: bytes) -> List[Tuple[str, bytes]]:
        msgs, i = [], 0
        while i + 24 <= len(data):
            if data[i:i+4] != self.magic:
                i += 1
                continue
            try:
                cmd = data[i+4:i+16].rstrip(b'\x00').decode()
                length = struct.unpack('<I', data[i+16:i+20])[0]
                payload = data[i+24:i+24+length]
                msgs.append((cmd, payload))
                i += 24 + length
            except:
                break
        return msgs

    def recv_all(self, sock: socket.socket, timeout: float = 5.0) -> bytes:
        sock.setblocking(False)
        data = b''
        end = time.time() + timeout
        while time.time() < end:
            try:
                r, _, _ = select.select([sock], [], [], 0.1)
                if r:
                    chunk = sock.recv(131072)
                    if chunk:
                        data += chunk
                    else:
                        break
            except:
                break
        sock.setblocking(True)
        return data

    # ============================================
    # SINCRONIZACIÓN COMPLETA DE HEADERS
    # ============================================

    def sync_headers(self, ip: str, max_rounds: int = 50) -> bool:
        """Sincroniza headers hasta el tip actual."""

        self.log(f"🔄 Sincronizando headers desde {ip}...")

        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.settimeout(30)
            sock.connect((ip, 8333))

            # Handshake
            sock.send(self.msg('version', self.version_payload(ip)))
            time.sleep(0.5)

            data = self.recv_all(sock, 3.0)
            peer_height = 0

            for cmd, payload in self.parse_msgs(data):
                if cmd == 'version':
                    if len(payload) >= 85:
                        peer_height = struct.unpack('<i', payload[81:85])[0]
                        self.log(f"   Peer está en altura: {peer_height:,}")
                    sock.send(self.msg('verack', b''))
                elif cmd == 'ping':
                    sock.send(self.msg('pong', payload[:8]))

            time.sleep(0.3)

            # Genesis hash como punto de partida
            genesis_hash = bytes.fromhex("000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f")[::-1]

            current_tip = genesis_hash
            total_headers = 0

            for round_num in range(max_rounds):
                # Construir block locator
                block_locator = [current_tip]

                sock.send(self.msg('getheaders', self.getheaders_payload(block_locator)))

                time.sleep(0.5)
                data = self.recv_all(sock, 5.0)

                headers_received = 0

                for cmd, payload in self.parse_msgs(data):
                    if cmd == 'headers':
                        count, offset = self.read_varint(payload, 0)

                        if count == 0:
                            self.log(f"   ✅ Sincronización completa!")
                            sock.close()
                            return True

                        for i in range(count):
                            if offset + 81 > len(payload):
                                break
                            header_data = payload[offset:offset+80]
                            self.all_headers.append(header_data)
                            current_tip = self.double_sha256(header_data)
                            offset += 81
                            headers_received += 1

                        total_headers += headers_received

                        if round_num % 10 == 0 or headers_received < 2000:
                            tip_hash = current_tip[::-1].hex()
                            self.log(f"   Round {round_num}: +{headers_received} headers (total: {total_headers:,}) tip: {tip_hash[:16]}...")

                    elif cmd == 'ping':
                        sock.send(self.msg('pong', payload[:8]))

                if headers_received < 2000:
                    # Ya llegamos al tip
                    self.log(f"   ✅ Llegamos al tip! Total: {total_headers:,} headers")
                    break

            sock.close()

            if self.all_headers:
                self.tip_header = self.all_headers[-1]
                self.tip_height = len(self.all_headers)
                return True

            return False

        except Exception as e:
            self.log(f"   ❌ Error: {e}")
            return False

    # ============================================
    # CREAR BLOQUE SOBRE TIP REAL
    # ============================================

    def create_injection_block(self, message: str = "KAORU") -> Block:
        """Crea bloque sobre el tip real."""

        prev_hash = self.double_sha256(self.tip_header)

        # Obtener bits del tip
        prev_bits = struct.unpack('<I', self.tip_header[72:76])[0]

        script = f"KAORU_BRIDGE_v9_{message}".encode()[:100]
        merkle = self.double_sha256(script)

        header = BlockHeader(
            version=0x20000000,
            prev_hash=prev_hash,
            merkle_root=merkle,
            timestamp=int(time.time()),
            bits=prev_bits,
            nonce=0
        )

        return Block(header=header, coinbase_script=script)

    def mine_block(self, block: Block, target_zeros: int = 4) -> bool:
        target = "0" * target_zeros
        start = time.time()

        for nonce in range(100_000_000):
            block.header.nonce = nonce
            h = block.header.hash_hex()

            if h.startswith(target):
                elapsed = time.time() - start
                self.log(f"   ✅ Minado! Hash: {h[:40]}...")
                self.log(f"      Nonce: {nonce:,} | Tiempo: {elapsed:.2f}s")
                return True

            if nonce % 2_000_000 == 0 and nonce > 0:
                self.log(f"      ... {nonce:,} hashes")

        return False

    # ============================================
    # PROPAGACIÓN
    # ============================================

    def propagate(self, ip: str, block: Block, duration: int = 60) -> Dict:
        result = {
            'ip': ip,
            'connected': False,
            'headers_sent': 0,
            'blocks_sent': 0,
            'getdata_received': False,
        }

        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.settimeout(15)
            sock.connect((ip, 8333))
            result['connected'] = True
            start = time.time()

            self.log("→ Conectado!")
            sock.send(self.msg('version', self.version_payload(ip)))

            inv_sent = False

            while time.time() - start < duration:
                data = self.recv_all(sock, 2.0)

                if not data:
                    time.sleep(0.2)
                    continue

                for cmd, payload in self.parse_msgs(data):
                    self.stats[f'recv_{cmd}'] += 1

                    if cmd == 'version':
                        try:
                            ua_len = payload[80]
                            agent = payload[81:81+ua_len].decode('ascii', errors='ignore')
                            self.log(f"← VERSION ({agent[:35]})")
                        except:
                            pass
                        sock.send(self.msg('verack', b''))
                        self.stats['sent_verack'] += 1

                    elif cmd == 'verack':
                        self.log("← VERACK ✓")

                        if not inv_sent:
                            time.sleep(0.2)
                            inv = b'\x01' + struct.pack('<I', self.MSG_BLOCK) + block.header.hash()
                            sock.send(self.msg('inv', inv))
                            self.stats['sent_inv'] += 1
                            inv_sent = True
                            self.log(f"→ INV (bloque {block.header.hash_hex()[:20]}...)")

                    elif cmd == 'ping':
                        sock.send(self.msg('pong', payload[:8]))
                        self.stats['sent_pong'] += 1
                        self.log("← PING → PONG")

                    elif cmd == 'sendcmpct':
                        sock.send(self.msg('sendcmpct', b'\x00' + struct.pack('<Q', 2)))

                    elif cmd == 'getheaders':
                        self.log("← GETHEADERS ⭐")

                        headers = b'\x01' + block.header.serialize() + b'\x00'
                        sock.send(self.msg('headers', headers))
                        self.stats['sent_headers'] += 1
                        result['headers_sent'] += 1
                        self.log("→ HEADERS (1)")

                    elif cmd == 'getdata':
                        self.log("← GETDATA ⭐⭐⭐ ¡¡¡PIDEN NUESTRO BLOQUE!!!")
                        result['getdata_received'] = True

                        count, offset = self.read_varint(payload, 0)

                        for _ in range(count):
                            if offset + 36 > len(payload):
                                break
                            inv_type = struct.unpack('<I', payload[offset:offset+4])[0]
                            inv_hash = payload[offset+4:offset+36]
                            offset += 36

                            if inv_hash == block.header.hash():
                                sock.send(self.msg('block', block.serialize()))
                                self.stats['sent_block'] += 1
                                result['blocks_sent'] += 1
                                self.log("→ BLOCK 📦📦📦 ¡¡ENVIADO!!")

                    elif cmd == 'reject':
                        try:
                            msg_len = payload[0]
                            msg_type = payload[1:1+msg_len].decode()
                            code = payload[1+msg_len]
                            reason_len = payload[2+msg_len]
                            reason = payload[3+msg_len:3+msg_len+reason_len].decode()
                            self.log(f"← REJECT ❌ {msg_type}: {reason}")
                        except:
                            self.log("← REJECT ❌")

                    elif cmd == 'inv':
                        count, _ = self.read_varint(payload, 0)
                        self.log(f"← INV ({count} items)")

                    elif cmd == 'wtxidrelay':
                        sock.send(self.msg('wtxidrelay', b''))

            sock.close()

        except Exception as e:
            result['error'] = str(e)

        return result

    def discover_nodes(self) -> List[str]:
        nodes = []
        for seed in ['seed.bitcoin.sipa.be', 'dnsseed.bluematt.me', 'seed.bitcoinstats.com']:
            try:
                nodes.extend(socket.gethostbyname_ex(seed)[2][:10])
            except:
                pass
        random.shuffle(nodes)
        return nodes

    # ============================================
    # EJECUCIÓN
    # ============================================

    def execute(self, num_peers: int = 3, sync_rounds: int = 5):
        print(f"""
╔════════════════════════════════════════════════════════════════════════════════════════╗
║                                                                                        ║
║    ██╗  ██╗ █████╗  ██████╗ ██████╗ ██╗   ██╗    ██╗   ██╗ █████╗     ██████╗          ║
║    ██║ ██╔╝██╔══██╗██╔═══██╗██╔══██╗██║   ██║    ██║   ██║██╔══██╗   ██╔═══██╗         ║
║    █████╔╝ ███████║██║   ██║██████╔╝██║   ██║    ██║   ██║╚██████║   ╚██████╔╝         ║
║    ██╔═██╗ ██╔══██║██║   ██║██╔══██╗██║   ██║    ╚██╗ ██╔╝ ╚═══██║    ╚═══██╗          ║
║    ██║  ██╗██║  ██║╚██████╔╝██║  ██║╚██████╔╝     ╚████╔╝  █████╔╝██╗██████╔╝          ║
║    ╚═╝  ╚═╝╚═╝  ╚═╝ ╚═════╝ ╚═╝  ╚═╝ ╚═════╝       ╚═══╝   ╚════╝ ╚═╝╚═════╝           ║
║                                                                                        ║
║                              TRUE TIP INJECTION                                        ║
║                      "Sincroniza hasta el tip REAL"                                    ║
║                                                                                        ║
╚════════════════════════════════════════════════════════════════════════════════════════╝
        """)

        # Descubrir nodos
        print("=" * 95)
        print("[FASE 1] 🌐 DESCUBRIENDO NODOS")
        print("=" * 95)

        nodes = self.discover_nodes()
        print(f"\n   Encontrados: {len(nodes)} nodos")

        # Sincronizar headers
        print("\n" + "=" * 95)
        print(f"[FASE 2] 🔗 SINCRONIZANDO HEADERS (esto puede tomar ~1-2 min)")
        print("=" * 95 + "\n")

        for ip in nodes[:3]:
            if self.sync_headers(ip, max_rounds=sync_rounds):
                break

        if not self.tip_header:
            print("\n   ❌ No se pudo sincronizar")
            return

        tip_hash = self.double_sha256(self.tip_header)[::-1].hex()
        print(f"\n   📋 Sincronización completada:")
        print(f"      Headers totales: {len(self.all_headers):,}")
        print(f"      Tip hash: {tip_hash}")

        # Crear bloque
        print("\n" + "=" * 95)
        print("[FASE 3] ⛏️  CREANDO BLOQUE SOBRE TIP REAL")
        print("=" * 95 + "\n")

        block = self.create_injection_block("REAL_TIP_INJECTION")

        print(f"   Bloque creado sobre altura ~{len(self.all_headers):,}")
        print(f"   prev_hash: {block.header.prev_hash[::-1].hex()[:48]}...")
        print(f"\n   ⛏️ Minando...")

        if self.mine_block(block, target_zeros=4):
            self.our_blocks.append(block)
            print(f"\n   ✅ Hash: {block.header.hash_hex()}")
        else:
            return

        # Propagar
        print("\n" + "=" * 95)
        print("[FASE 4] 📡 PROPAGANDO")
        print("=" * 95)

        results = []

        for ip in nodes[:num_peers]:
            print(f"\n   ╔{'═'*85}╗")
            print(f"   ║  📡 {ip:77} ║")
            print(f"   ╠{'═'*85}╣")

            result = self.propagate(ip, block, 60)
            results.append(result)

            print(f"   ╠{'═'*85}╣")
            print(f"   ║  Headers: {result['headers_sent']} | Blocks: {result['blocks_sent']} | GETDATA: {'✅' if result['getdata_received'] else '❌':40} ║")
            print(f"   ╚{'═'*85}╝")

        # Resultados
        print("\n" + "=" * 95)
        print("                              📊 RESULTADOS")
        print("=" * 95)

        total_blocks = sum(r['blocks_sent'] for r in results)
        getdata_count = sum(1 for r in results if r['getdata_received'])

        print(f"""
   ╔═══════════════════════════════════════════════════════════════════════════════════════╗
   ║                                                                                       ║
   ║   Headers sincronizados:    {len(self.all_headers):>8,}                                              ║
   ║   Tip height aproximado:    {len(self.all_headers):>8,}                                              ║
   ║   GETDATA recibidos:        {getdata_count:>8}                                              ║
   ║   BLOQUES ENVIADOS:         {total_blocks:>8}                                              ║
   ║                                                                                       ║
   ║   {'✅ GETDATA RECIBIDO - BLOQUE ENVIADO!' if total_blocks > 0 else '❌ Sin GETDATA todavía':60}              ║
   ║                                                                                       ║
   ╚═══════════════════════════════════════════════════════════════════════════════════════╝
        """)

        return results


if __name__ == "__main__":
    bridge = KaoruBridgeV9()
    # sync_rounds controla cuántos batches de 2000 headers pedimos
    # 5 rounds = ~10,000 headers (no el tip completo, pero más cerca)
    bridge.execute(num_peers=3, sync_rounds=5)


╔════════════════════════════════════════════════════════════════════════════════════════╗
║                                                                                        ║
║    ██╗  ██╗ █████╗  ██████╗ ██████╗ ██╗   ██╗    ██╗   ██╗ █████╗     ██████╗          ║
║    ██║ ██╔╝██╔══██╗██╔═══██╗██╔══██╗██║   ██║    ██║   ██║██╔══██╗   ██╔═══██╗         ║
║    █████╔╝ ███████║██║   ██║██████╔╝██║   ██║    ██║   ██║╚██████║   ╚██████╔╝         ║
║    ██╔═██╗ ██╔══██║██║   ██║██╔══██╗██║   ██║    ╚██╗ ██╔╝ ╚═══██║    ╚═══██╗          ║
║    ██║  ██╗██║  ██║╚██████╔╝██║  ██║╚██████╔╝     ╚████╔╝  █████╔╝██╗██████╔╝          ║
║    ╚═╝  ╚═╝╚═╝  ╚═╝ ╚═════╝ ╚═╝  ╚═╝ ╚═════╝       ╚═══╝   ╚════╝ ╚═╝╚═════╝           ║
║                                                                                        ║
║                              TRUE TIP INJECTION                                        ║
║                      "Sincroniza hasta el tip REAL"                                    